# User Defined Functions

If you can use the built-in functions in `sql.functions`, you certainly should use them as they less likely to have bugs and can be optimized by Catalyst.

However, the built-in functions don't contain everything you need, and sometimes you have to write custom functions, known as User Defined Functions.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:</br>

* Create User Defined Functions (UDFs)
* Articulate performance advantages of Vectorized UDFs in Python

In [0]:
%run "./Includes/Classroom-Setup"

In [0]:
filePath = "dbfs:/mnt/training/airbnb/sf-listings/sf-listings-2019-03-06.csv"
rawDF = spark.read.csv(filePath, header=True, inferSchema=True, multiLine=True, escape='"')
display(rawDF)

id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
958,https://www.airbnb.com/rooms/958,20190306152813,2019-03-06,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under construction and there is the possibility of noise from 7am - 5pm. Our rates are discounted during this time period. Our bright garden unit overlooks a grassy backyard area with fruit trees and native plants. It is an oasis in a big city. The apartment comfortably fits a couple or small family. It is located on a cul de sac street that ends at lovely Duboce Park.,"Newly remodeled, modern, and bright garden unit in historic Victorian home. *New fixtures and finishes. *Organic cotton sheets and towels. *Zero VOC and non-toxic Yolo paint. *Organic and fair-trade teas, fresh local ground coffee. *Local art on walls. *Sofa bed and Queen bed are in the same room. More of a petite apartment with a separate room for dining and kitchen.","New update: the house next door is under construction and there is the possibility of noise from 7am - 5pm. Our rates are discounted during this time period. Our bright garden unit overlooks a grassy backyard area with fruit trees and native plants. It is an oasis in a big city. The apartment comfortably fits a couple or small family. It is located on a cul de sac street that ends at lovely Duboce Park. Newly remodeled, modern, and bright garden unit in historic Victorian home. *New fixtures and finishes. *Organic cotton sheets and towels. *Zero VOC and non-toxic Yolo paint. *Organic and fair-trade teas, fresh local ground coffee. *Local art on walls. *Sofa bed and Queen bed are in the same room. More of a petite apartment with a separate room for dining and kitchen. *Full access to patio and backyard (shared with us and our dog who live upstairs) *Beautiful garden with fruit trees, native plants and lawn *Washer and dryer *Children's toys *Charcoal grill A family of 4 lives upstairs",none,"*Quiet cul de sac in friendly neighborhood *Steps away from grassy park with 2 playgrounds and Recreational Center *Very family-friendly neighborhood *Quaint shops, grocery stores and restaurants all within a 5-10 minute walk","Due to the fact that we have children and a dog, we are up early 7-8am and their footsteps or paws can be heard from the apartment. Our place is ide

Let's select the columns`id`, `host_name`, `bedrooms`, `neighbourhood_cleansed`, and `price`, as well as filter out all the rows that contain nulls.

In [0]:
airbnbDF = rawDF.select("id", "host_name", "bedrooms", "neighbourhood_cleansed", "price").dropna()
display(airbnbDF)

id,host_name,bedrooms,neighbourhood_cleansed,price
958,Holly,1,Western Addition,$170.00
5858,Philip And Tania,2,Bernal Heights,$235.00
7918,Aaron,1,Haight Ashbury,$65.00
8142,Aaron,1,Haight Ashbury,$65.00
8339,Rosy,2,Western Addition,$785.00
8567,Sandy,2,Western Addition,$255.00
8739,Ivan & Wendy,1,Mission,$139.00
9225,Gae,1,Potrero Hill,$135.00
10251,Roman & Sarah,2,Mission,$265.00
10820,Bernat,3,Haight Ashbury,$177.00


### User Defined Functions

We've seen many built-in functions (e.g. `avg`, `lit`, `col`, etc.). However, sometimes you might need a specific function that is not provided, so let's look at how to define your own **User Defined Function**.

For example, let's say we want to get the first initial from our `host_name` field. Let's start by writing that function in local Python/Scala.

In [0]:
def firstInitialFunction(name):
  return name[0]

firstInitialFunction("Jane")

Out[7]: 'J'

Now we have to define a UDF that wraps the function. This will serialize our function and send it to the executors so that we can use it in our DataFrame.

In [0]:
firstInitialUDF = udf(firstInitialFunction)

In [0]:
from pyspark.sql.functions import col
display(airbnbDF.select(firstInitialUDF(col("host_name"))))

firstInitialFunction(host_name)
H
P
A
A
R
S
I
G
R
B


We can also create a UDF using `spark.sql.register`, which will create the UDF in the SQL namespace.

In [0]:
airbnbDF.createOrReplaceTempView("airbnbDF")

spark.udf.register("sql_udf", firstInitialFunction)

Out[11]: <function __main__.firstInitialFunction(name)>

In [0]:
%sql
select sql_udf(host_name) as firstInitial from airbnbDF

firstInitial
H
P
A
A
R
S
I
G
R
B


### Decorator Syntax [Python Only]

Alternatively, you can define a UDF using decorator syntax in Python with the dataType the function will return. 

However, you cannot call the local Python function anymore (e.g. `decoratorUDF("Jane")` will not work)

In [0]:
%python
# Our input/output is a sting
@udf("string")
def decoratorUDF(name):
  return name[0]

In [0]:
display(airbnbDF.select(decoratorUDF(col("host_name"))))

decoratorUDF(host_name)
H
P
A
A
R
S
I
G
R
B


UDFs provide even more functionality, but it is best to use a built in function wherever possible.

**UDF Drawbacks:**
* UDFs cannot be optimized by the Catalyst Optimizer
* The function **has to be serialized** and sent out to the executors
* In the case of Python, there is even more overhead - we have to **spin up a Python interpreter** on every Executor to run the UDF (e.g. Python UDFs much slower than Scala UDFs)

### Vectorized UDF

As of Spark 2.3, there are Vectorized UDFs available in Python to help speed up the computation.

* [Blog post](https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html)
* [Documentation](https://spark.apache.org/docs/latest/sql-programming-guide.html#pyspark-usage-guide-for-pandas-with-apache-arrow)

![Benchmark](https://databricks.com/wp-content/uploads/2017/10/image1-4.png)

Vectorized UDFs utilize Apache Arrow to speed up computation. Let's see how that helps improve our processing time.

[Apache Arrow](https://arrow.apache.org/), is an in-memory columnar data format that is used in Spark to efficiently transfer data between JVM and Python processes. See more [here](https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html).

In [0]:
%python
from pyspark.sql.functions import pandas_udf

# We have a string input/output
@pandas_udf("string")
def vectorizedUDF(name):
  return name.str[0]

In [0]:
display(airbnbDF.select(vectorizedUDF(col("host_name"))))

vectorizedUDF(host_name)
H
P
A
A
R
S
I
G
R
B


We can also register these Vectorized UDFs to the SQL namespace.

In [0]:
%python
spark.udf.register("sql_vectorized_udf", vectorizedUDF)

Out[16]: <function __main__.vectorizedUDF(name)>